>[Modeling](#scrollTo=PMnoJMML6aiC)

>>[Data Prep and Cleaning](#scrollTo=aoyYaNFL8Z53)

>>[Train and Test Split](#scrollTo=pQYAAiTy8xAm)

>>[Set Baseline](#scrollTo=CDoDEYVh9Dln)

>>[Logisitic Regression](#scrollTo=O6vEyvtJ9SM9)



# Modeling
**Amber Cash**

The purpose of this notebook is to document the modeling process for my group's (STAR Analytics) submission for the Home Credit Kaggle competition. Below you will find my contribition, which was testing logistic regression models and assessing if they pass the performance benchmarks sets by themajority classifer model.



## Data Prep and Cleaning

To start, let's load in all packages and prepare the data for analysis. This includes aggregating data, joining our 3 data sets, and handling missing values by replacing them with the median value.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# Load data
train_data = pd.read_csv('application_train.csv')
bureau_data = pd.read_csv('bureau.csv')
previous_app_data = pd.read_csv('previous_application.csv')

# Aggregate bureau data
bureau_agg = bureau_data.groupby('SK_ID_CURR').agg({
    'AMT_CREDIT_SUM': ['sum', 'mean'],
    'AMT_CREDIT_SUM_DEBT': ['sum', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['sum', 'mean'],
    'CNT_CREDIT_PROLONG': ['sum']
})
bureau_agg.columns = ['_'.join(col).upper() for col in bureau_agg.columns]
bureau_agg.reset_index(inplace=True)

# Aggregate previous application data
previous_app_agg = previous_app_data.groupby('SK_ID_CURR').agg({
    'AMT_APPLICATION': ['mean', 'max'],
    'AMT_CREDIT': ['mean', 'max'],
    'DAYS_DECISION': ['mean'],
    'NAME_CONTRACT_STATUS': ['count']
})
previous_app_agg.columns = ['_'.join(col).upper() for col in previous_app_agg.columns]
previous_app_agg.reset_index(inplace=True)

# Merge aggregated features with training data
train_data = train_data.merge(bureau_agg, on='SK_ID_CURR', how='left')
train_data = train_data.merge(previous_app_agg, on='SK_ID_CURR', how='left')

# Select features and target
features = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH',
            'AMT_CREDIT_SUM_SUM', 'AMT_CREDIT_SUM_MEAN', 'AMT_CREDIT_SUM_DEBT_SUM', 'AMT_CREDIT_SUM_DEBT_MEAN',
            'AMT_APPLICATION_MEAN', 'AMT_APPLICATION_MAX', 'AMT_CREDIT_MEAN', 'AMT_CREDIT_MAX']
target = 'TARGET'

X = train_data[features]
y = train_data[target]

# Handle missing values (simple imputation)
X.fillna(X.median(), inplace=True)

## Train and Test Split

Now, let's split our data into train and test split to use in our model training.

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

## Set Baseline

Now, let's train our baseline - a majority classifier. We will use the performance metrics from this model as a benchmark while testing other models.

In [5]:
# Baseline model: Majority class classifier (predicts the most frequent class)
majority_class = y_train.mode()[0]
y_majority_pred = np.full_like(y_val, majority_class)
majority_accuracy = accuracy_score(y_val, y_majority_pred)
print(f'Majority Class Accuracy: {majority_accuracy:.4f}')

Majority Class Accuracy: 0.9193


## Logisitic Regression

Finally, let's train two logistic regression models (one default and one interaction term) to see if we can beat the beanchmark performance metrics.

In [4]:
# Perform k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Fit logistic regression model with cross-validation
model = LogisticRegression()
cross_val_auc = cross_val_score(model, X_train_scaled, y_train, cv=kf, scoring='roc_auc')
print(f'Cross-Validation AUC: {cross_val_auc.mean():.4f}')

# Train logistic regression model on full training set
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_val_scaled)
y_prob = model.predict_proba(X_val_scaled)[:, 1]

# Evaluate model
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)

print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Validation ROC AUC Score: {roc_auc:.4f}')

Cross-Validation AUC: 0.6288
Validation Accuracy: 0.9192
Validation ROC AUC Score: 0.6243


We can see that a basic logistic regression model did not outperform our benchmark. Let's try introducing an interaction term to improve accuracy.

In [6]:
# Fit logistic regression models with interaction terms
X_train['INCOME_CREDIT_RATIO'] = X_train['AMT_INCOME_TOTAL'] / X_train['AMT_CREDIT']
X_val['INCOME_CREDIT_RATIO'] = X_val['AMT_INCOME_TOTAL'] / X_val['AMT_CREDIT']

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

model_interaction = LogisticRegression()
model_interaction.fit(X_train_scaled, y_train)

# Make predictions
y_pred_interaction = model_interaction.predict(X_val_scaled)
y_prob_interaction = model_interaction.predict_proba(X_val_scaled)[:, 1]

# Evaluate interaction model
accuracy_interaction = accuracy_score(y_val, y_pred_interaction)
roc_auc_interaction = roc_auc_score(y_val, y_prob_interaction)

print(f'Validation Accuracy with Interaction: {accuracy_interaction:.4f}')
print(f'Validation ROC AUC Score with Interaction: {roc_auc_interaction:.4f}')


Validation Accuracy with Interaction: 0.9192
Validation ROC AUC Score with Interaction: 0.6243


Again, we see this logistic regression fail to outperform the benchmark.

Based on these results, we can assume that logistic regression is not the best fit for this data. It is likely that this model cannot handle the complexities of the Home Credit data. For that reason, STAR Analytics ultimately used an XG Boost model instead. That model was able to outperform the benchmark, providing promising predictive power.